# Task 4.2: Dimensionality Reduction

## 4.2.1 - Principal Component Analysis (PCA)

Data is one of the main drivers behind most of Machine Learning applications. Analysing the data prior to any usage is of crucial importance, because data can be complicated and it may be challenging to understand what it all means and which parts actually carry information.

Dimensionality reduction is a technique which helps us gain a better macro-level understanding of the data by reducing the number of features that describe a dataset such that only the most information-rich parts remain.

Principal Component Analysis (PCA) is a simple yet powerful technique used for dimensionality reduction. Through it, we can directly decrease the number of feature variables, thereby narrowing down the important features and saving on computations. From a high-level view PCA has three main steps:

1. Compute the **covariance matrix** of the data
2. Compute the **eigen values** and **vectors** of this covariance matrix
3. Use the eigen values and vectors to select only the most important feature dimensions that carry enough information to approximate the original data.


If you are interested to know more about the PCA steps, check [this](https://plot.ly/python/v3/ipython-notebooks/principal-component-analysis/) useful tutorial in NumPy.


The entire process will be illustrated in the script below, using the same [Meteonorm](https://meteonorm.com/en/download) data.

In [ ]:
#@title Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#@title Load data
df1 = pd.read_table('./weather-data/Bern-hour.dat', sep=',', header=None)
df2 = pd.read_table('./weather-data/Johannesburg-hour.dat', sep=',', header=None)
df3 = pd.read_table('./weather-data/SanFrancisco-hour.dat', sep=',', header=None)
df4 = pd.read_table('./weather-data/Perth-hour.dat', sep=',', header=None)
df5 = pd.read_table('./weather-data/Brasilia-hour.dat', sep=',', header=None)

In [ ]:
#@title Helper function
def create_dataframe(dataframe, cls):
    
    import datetime as dt
    
    dataframe.columns = ['year', 'month', 'day', 'hour', 'global radiation', 'diffuse radiation', 
                         'temp', 'wind speed','relative humidity', 'cloud cover', 'precipitation']
    
    datetime = dataframe.loc[:, 'year':'hour']
    
    # the original data has hours in the 1 --> 24 format, but datetime accepts only 0 --> 23
    datetime['hour'] = datetime['hour'] - 1
    
    datetime['DateTime'] = datetime.apply(lambda row: dt.datetime(row.year, row.month, row.day, row.hour), axis=1)
    datetime['DateTime'] = pd.to_datetime(datetime.DateTime)
    dataframe.index = pd.DatetimeIndex(datetime.DateTime)
    
    # include the class column for each city
    dataframe['class'] = np.full(shape=dataframe.shape[0], fill_value=cls)
    
    # delete the first four columns, they are not needed now that there is a DateTimeIndex
    dataframe = dataframe.drop(['year', 'month', 'day', 'hour'], axis=1)
    
    return dataframe


In [ ]:
#@title Preprocess data frames
df_bern = create_dataframe(df1, cls=0)
df_perth = create_dataframe(df4, cls=1)
df_johannesburg = create_dataframe(df2, cls=2)
df_sanfrancisco = create_dataframe(df3, cls=3)
df_brasilia = create_dataframe(df5, cls=4)

## 4.2.1.1. Applying PCA to two cities

As an example, we have decided to apply the PCA to the cities of Perth and Bern, since they belong to the Southern and Northern hemisphere, respectively. Owing to the tilt of Earth's rotation relative to the Sun and the ecliptic plane, the seasons are basically inverted between the two hemispheres. 

Therefore, we expect to find clear differences in weather data by extracting only the four months in which it is winter in Bern and summer in Perth, i.e. January, February, November and December. Via PCA, we can reduce the dimensionality of the dataset by projecting the whole data **into the few dimensions that are necessary to discriminated between the two hemispheres**. 

In [ ]:
# we can take the desired period by dropping the months in the middle (i.e. March to October)

df_perth_summer = df_perth.drop(df_perth.loc['2005-03-01 00:00:00':'2005-10-31 23:00:00', :].index, axis=0)

df_perth_summer

In [ ]:
df_bern_winter = df_bern.drop(df_bern.loc['2005-03-01 00:00:00':'2005-10-31 23:00:00', :].index, axis=0)

df_bern_winter

In [ ]:
# we now combined the data of both cities for our PCA analysis
df_pca = pd.concat([df_bern_winter, df_perth_summer], keys=['Bern','Perth'], axis=0, join='inner')

df_pca

First, we take all the weather parameters (X) as inputs to the PCA algorithm. The target (y) is the city identifier, *class*.

In [ ]:
X = df_pca.loc[:, 'global radiation':'precipitation'].values
y = df_pca.loc[:, 'class'].values

PCA returns a sub-space that maximizes the variance along the feature vectors. Therefore, in order to properly measure the variance of those feature vectors, they must be equally balanced. To do this, we first normalise our data to have zero-mean and unit-variance such that each feature will be weighted equally in our calculations. 

This is done by using the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) module in **scikit-learn**.

In [ ]:
# the data is scaled 

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

scikit-learn implements a module for computing the PCA: `fit_transform()`. You can specify the desired number of principal components or you can take the ones that hold most of the variance (or information) about the dataset. Depending on the problem and the data, the first 2 or 3 principal components can be sufficient.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
Y = pca.fit_transform(X_std)
print(pca.explained_variance_ratio_)



How much of the variance is explained by the first two principal components?
Let's see whether or not it is enough to clearly discriminated between Bern and Perth. 

Here is an example of how the data looks like for the first principal component.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.scatter(Y[:, 0][y == 0], np.zeros((Y[y == 0].shape[0], 1)), marker='o', s=50, label='Bern')
ax.scatter(Y[:, 0][y == 1], np.zeros((Y[y == 1].shape[0], 1)), marker='x', s=10, label='Perth')

ax.set_xlim([-6, 6])
ax.set_ylim([-1, 1])

ax.set_xlabel('Principal Component 1')
ax.set_title('Scatter plot for the cities of Bern and Perth - Jan, Feb, Nov and Dec')
plt.legend()
plt.show()

You can see that there is a big overlap between the two classes (cities). 

**[TO DO]:** What happens if you plot the first two principal components?

In [ ]:
###########################
# Task: 
#   Scatterplot with the first two principal components
###########################


### TO DO


**[Optional]:** Try to make a 3-D scatterplot using the three principal 

In [ ]:
# how to generate 3d figures using matplotlib 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

### TO DO

## 4.2.2 - Non Linear Dimensionality Reduction 


What happens when your data are linearly inseparable? Was the data that we previously applied PCA to actually linearly separable? Remember that PCA is a linear projection technique.



### 4.2.2.1 Kernel Principal Component Analysis (K-PCA)

The first approach we'll look at is Kernel PCA which uses a non-linear kernel function with standard PCA to achieve non-linear dimensionality reduction. In practice, it projects the linearly inseparable data into a higher dimensional space that can be linearly separated using PCA. The main steps of KPCA are:
1. Select a non-linear kernel mapping
2. Construct the normalized kernel matrix
3. Find the eigen values and vectors of this matrix
4. For each data point, use the eigen values and vectors to obtain its principal components.

Most common kernel functions are Gaussian, Sigmoid and Polynomial. You can learn more details about Kernel PCA [here](http://axon.cs.byu.edu/~martinez/classes/778/Papers/KernelPCA.pdf). You might also find it useful to read about [Kernel SVM](https://scikit-learn.org/stable/modules/svm.html) since it has many conceptual similarities to  Kernel PCA. 


We use the same dataset as in PCA example. We will apply the Kernal PCA to the cities of Perth and Johannesburg and we are going to use only January, February, November and December which it is summer both in Perth and  Johannesburg. 

In [ ]:
#@title Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#@title Load and preprocess data as in PCA but just for 'Perth' and 'Johannesburg'
df2 = pd.read_table('./weather-data/Johannesburg-hour.dat', sep=',', header=None)
df4 = pd.read_table('./weather-data/Perth-hour.dat', sep=',', header=None)

# helper function
def create_dataframe(dataframe, cls):
    
    import datetime as dt
    
    dataframe.columns = ['year', 'month', 'day', 'hour', 'global radiation', 'diffuse radiation', 
                         'temp', 'wind speed','relative humidity', 'cloud cover', 'precipitation']
    
    datetime = dataframe.loc[:, 'year':'hour']
    
    # the original data has hours in the 1 --> 24 format, but datetime accepts only 0 --> 23
    datetime['hour'] = datetime['hour'] - 1
    
    datetime['DateTime'] = datetime.apply(lambda row: dt.datetime(row.year, row.month, row.day, row.hour), axis=1)
    datetime['DateTime'] = pd.to_datetime(datetime.DateTime)
    dataframe.index = pd.DatetimeIndex(datetime.DateTime)
    
    # include the class column for each city
    dataframe['class'] = np.full(shape=dataframe.shape[0], fill_value=cls)
    
    # delete the first four columns, they are not needed now that there is a DateTimeIndex
    dataframe = dataframe.drop(['year', 'month', 'day', 'hour'], axis=1)
    
    return dataframe

df_perth = create_dataframe(df4, cls=1)
df_johannesburg = create_dataframe(df2, cls=2)
# we can take the desired period by dropping the months in the middle (i.e. March to October)

df_perth_summer = df_perth.drop(df_perth.loc['2005-03-01 00:00:00':'2005-10-31 23:00:00', :].index, axis=0)
df_johannesburg_summer = df_johannesburg.drop(df_johannesburg.loc['2005-03-01 00:00:00':'2005-10-31 23:00:00', :].index, axis=0)

# we now combined the data of both cities for our PCA analysis
df_pca = pd.concat([df_perth_summer,df_johannesburg_summer], keys=['Perth','Johannesburg'], axis=0, join='inner')

df_pca

X = df_pca.loc[:, 'global radiation':'precipitation'].values
y = df_pca.loc[:, 'class'].values

# the data is scaled 

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)


Similar to PCA, scikit-learn provides `fit_transform()` for calculating Kernel PCA. You can specify different arguments such as the number of principal components, the kernel function and parameters related to them. 


In [ ]:
from sklearn.decomposition import KernelPCA

kpca =  KernelPCA(kernel='rbf')
Y_k = kpca.fit_transform(X_std)


Compute the explained variance and ratio since kernel PCA does not have a built-in function.

In [ ]:
explained_variance = np.var(Y_k, axis=0)
explained_variance_ratio = explained_variance / np.sum(explained_variance)
explained_variance_ratio

 **[TO DO]:** Use PCA to separate the data and plot the first three principal components. 

In [ ]:
###########################
# Task: 
#   Use PCA for dimensionality reduction
###########################


### TO DO

 **[TO DO]:** Experiment with the following kernel functions such as `poly`, `sigmoid`, `cosine`, `precomputed`. Visualize the results of each kernel option. 

In [ ]:
###########################
# Task: 
#   Use kernel=poly, cosine, precomputed in KernelPCA 
###########################


### TO DO

**[TO DO]:** What happens when you select a linear kernel? Can you compare the results to standard PCA?

In [ ]:
###########################
# Task: 
#   Use kernel=linear in KernelPCA 
###########################


### TO DO

### 4.2.2.2 t-Distributed Stochastic Neighbor Embedding (t-SNE)

t-Distributed Stochastic Neighbor Embedding (t-SNE) is an non-linear approach for dimensionality reduction used for exploring and visualizing high-dimensional data. It uses gradient descent to minimize the Kullback-Leibler divergence between the distribution that measures pairwise similarities of the original high-dimensional inputs and the distribution that measures pairwise similarities of the corresponding embedded in a lower-dimension points.

You can find a summary of the algorithmic steps [here](https://uk.mathworks.com/help/stats/t-sne.html) or more details in the [original paper](https://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf).

There are quite few hyperparameters to control during optimization process (e.g. perplexity, learning rate, number of iterations etc.). You can find a very informative guideon how to deal with different situations on [How to Use t-SNE Effectively](https://distill.pub/2016/misread-tsne/ ).

The `sklearn.manifold` module implements the t-distributed Stochastic Neighbor Embedding. We apply the t-SNE on the data we defined in 2.2.1.

In [ ]:
from sklearn import manifold

tsne = manifold.TSNE(n_components=2, init='random',
                     random_state=0, perplexity=5)
Y = tsne.fit_transform(X)
Y.shape

**[TO DO]:** Visualize the results.

In [ ]:
###########################
# Task: 
#   3D Scatterplot with the transformed output of t-SNE
###########################


### TO DO


**[TO DO]:** Increase the perplexity a lot. What do you notice? What happens when it is higher that the data points? You can try other changes in the parameters.

In [ ]:
###########################
# Task: 
#   Change perplexity parameter
###########################


### TO DO


**[OPTIONAL]:** Compare t-SNE results to PCA. What are the advantages and disadvantages of each approach? What happens if you use PCA before t-SNE?

In [ ]:
###########################
# Task: 
#   Use PCA for dimensionality reduction
###########################


### TO DO

In [ ]:
###########################
# Task: 
#   3D Scatterplot with the results of PCA
###########################


### TO DO
